In [42]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")

In [2]:
mpl.rc('font', family='NanumGothic') # 폰트 설정
mpl.rc('axes', unicode_minus=False) # 유니코드에서 음수 부호 설정

# 차트 스타일 설정
sns.set(font="NanumGothic", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc("figure", figsize=(10,8))



In [ ]:
## 

In [5]:
news_data = fetch_20newsgroups(subset='all',random_state=156)

In [10]:
news_data['data'][0]

'From: egreen@east.sun.com (Ed Green - Pixel Cruncher)\nSubject: Re: Observation re: helmets\nOrganization: Sun Microsystems, RTP, NC\nLines: 21\nDistribution: world\nReply-To: egreen@east.sun.com\nNNTP-Posting-Host: laser.east.sun.com\n\nIn article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:\n> \n> The question for the day is re: passenger helmets, if you don\'t know for \n>certain who\'s gonna ride with you (like say you meet them at a .... church \n>meeting, yeah, that\'s the ticket)... What are some guidelines? Should I just \n>pick up another shoei in my size to have a backup helmet (XL), or should I \n>maybe get an inexpensive one of a smaller size to accomodate my likely \n>passenger? \n\nIf your primary concern is protecting the passenger in the event of a\ncrash, have him or her fitted for a helmet that is their size.  If your\nprimary concern is complying with stupid helmet laws, carry a real big\nspare (you can put a big or small 

In [6]:
news_data.keys()  #dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [24]:
print(news_data['target_names'][:5],"...")
print(news_data['target'][:5],"...")

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware'] ...
[ 8  8 12 10  6] ...


* target : 0 ~ 19

In [28]:
print("target 분포도")
pd.Series(news_data['target']).value_counts().sort_index()[:5]

target 분포도


0    799
1    973
2    985
3    982
4    963
dtype: int64

### train_test_split

In [30]:
# train set, 내용 외 정보 제거
train_news= fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

# test set, 내용 외 정보 제거
test_news= fetch_20newsgroups(subset='test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target

print(f'학습 데이터 크기 {len(train_news.data)} , 테스트 데이터 크기 {len(test_news.data)}')  #학습 데이터 크기 11314 , 테스트 데이터 크기 7532

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


### 피처 벡터화

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorization: train
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

# Count Vectorization: test
X_test_cnt_vect = cnt_vect.transform(X_test)

print(f"학습 데이터 Text의 CountVectorizer Shape: {X_train_cnt_vect.shape}")

학습 데이터 Text의 CountVectorizer Shape: (11314, 101631)


### 1차 점수 : ML

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV


# LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"CountVectorized + Logistic Regression 예측 정확도: {lr_acc:.3f}")

CountVectorized + Logistic Regression 예측 정확도: 0.605


### 2차 점수 :  TF-IDF 벡터화 : 0.674

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization: train
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# TF-IDF Vectorization: test
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"TF-IDF 정확도: {lr_acc:.3f}")  # 0.674

TF-IDF 정확도: 0.674


### 3차 점수 : TF-IDF 튜닝 : 0.692
* stop_words='english' 
* ngram_range=(1,2)

In [39]:
# TF-IDF Vectorization: train
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300 )
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)

# TF-IDF Vectorization: test
X_test_tfidf_vect = tfidf_vect.transform(X_test)

# LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"TF-IDF 튜닝 정확도: {lr_acc:.3f}")       #0.692

TF-IDF 튜닝 정확도: 0.692


### 4차 점수 : Model 튜닝 : 실행불가... 코랩으로 0.70

In [ ]:
# 최적 하이퍼 파라미터: C
params = { "C":[0.01, 0.1, 1, 5, 10] }
cv_model = GridSearchCV(lr_clf ,param_grid = params , cv=3 , scoring='accuracy' , verbose=1 )
cv_model.fit(X_train_tfidf_vect , y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

# 최적 C 값으로 학습된 grid_cv로 예측/평가
pred = cv_model.predict(X_test_tfidf_vect)
lr_acc = accuracy_score(y_test, pred)

print(f"CV 정확도: {lr_acc:.3f}")

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


### 5차 점수 : pipeline + GridSearchCV

* pipeline : TfidfVectorizer + LogisticRegression

In [ ]:
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
lr_acc = accuracy_score(y_test, pred)
print(f"pipeline 정확도: {lr_acc:.3f}")

*  pipeline +  GridSearchCV

In [45]:
# Pipeline 파라미터
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# GridSearchCV + Pipeline
params = { 'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
           'tfidf_vect__max_df': [100, 300, 700],
           'lr_clf__C': [1,5,10]
}
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv=3 , scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_ , grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
lr_acc = accuracy_score(y_test, pred)

print(f'Pipeline + CV 정확도: {lr_acc:.3f}')